In [1]:
import pandas as pd
import numpy as np
import json
import os
import random
import string
import re
import matplotlib.pyplot as plt
import pickle

from pathlib import Path
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from catboost import CatBoostRegressor
from catboost import Pool
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error,mean_squared_log_error
import optuna.integration.lightgbm as lgbo

import umap
pd.set_option('display.max_columns', 2000)

SEED = 777



In [2]:
df_train = pd.read_pickle('../data/train/full_comp.pickle')
df_train = df_train[df_train['cites'].isnull() == False].reset_index(drop=True)
df_train['cites'] = np.log1p(df_train['cites'])
df_train.shape


(15117, 2002)

In [3]:
df_train.head(1)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date_x,authors_parsed,doi_cites,cites,doi_id,pub_publisher,pub_journals,pub_dois,update_date_y,first_created_date,last_created_date,update_year,first_created_year,last_created_year,update_month,first_created_month,last_created_month,update_ym,first_created_ym,last_created_ym,update_day,first_created_day,last_created_day,update_date_unixtime,first_created_unixtime,last_created_unixtime,diff_update_date_unixtime,diff_created_unixtime,num_created,update_date_days,first_created_days,last_created_days,diff_created_days,rate_created_days,author_first,author_num,pred_doi_cites,category_main_detail,category_main,acc-phys_x,adap-org_x,alg-geom_x,ao-sci_x,astro-ph_x,atom-ph_x,bayes-an_x,chao-dyn_x,chem-ph_x,cmp-lg_x,comp-gas_x,cond-mat_x,cs_x,dg-ga_x,econ_x,eess_x,funct-an_x,gr-qc_x,hep-ex_x,hep-lat_x,hep-ph_x,hep-th_x,math_x,math-ph_x,mtrl-th_x,nlin_x,nucl-ex_x,nucl-th_x,patt-sol_x,physics_x,plasm-ph_x,q-alg_x,q-bio_x,q-fin_x,quant-ph_x,solv-int_x,stat_x,supr-con_x,acc_x,adap_x,alg_x,ao_x,astro_x,atom_x,bayes_x,chao_x,chem_x,cmp_x,comp_x,cond_x,cs_y,dg_x,econ_y,eess_y,funct_x,gr_x,hep_x,math_y,mtrl_x,nlin_y,nucl_x,patt_x,physics_y,plasm_x,q_x,quant_x,solv_x,stat_y,supr_x,category_name_parent_main_unique,category_name_parent_unique,category_name_unique,acc-phys_y,adap-org_y,alg-geom_y,ao-sci_y,astro-ph_y,atom-ph_y,bayes-an_y,chao-dyn_y,chem-ph_y,cmp-lg_y,comp-gas_y,cond-mat_y,dg-ga_y,funct-an_y,gr-qc_y,hep-ex_y,hep-lat_y,hep-ph_y,hep-th_y,math-ph_y,mtrl-th_y,nucl-ex_y,nucl-th_y,patt-sol_y,plasm-ph_y,q-alg_y,q-bio_y,q-fin_y,quant-ph_y,solv-int_y,supr-con_y,acc_y,adap_y,alg_y,ao_y,astro_y,atom_y,bayes_y,chao_y,chem_y,cmp_y,comp_y,cond_y,cs,dg_y,econ,eess,funct_y,gr_y,hep_y,math,mtrl_y,nlin,nucl_y,patt_y,physics,plasm_y,q_y,quant_y,solv_y,stat,supr_y,astro-ph.co,astro-ph.ep,astro-ph.ga,astro-ph.he,astro-ph.im,astro-ph.sr,cond-mat.dis-nn,cond-mat.mes-hall,cond-mat.mtrl-sci,cond-mat.other,cond-mat.quant-gas,cond-mat.soft,cond-mat.stat-mech,cond-mat.str-el,cond-mat.supr-con,cs.ai,cs.ar,cs.cc,cs.ce,cs.cg,cs.cl,cs.cr,cs.cv,cs.cy,cs.db,cs.dc,cs.dl,cs.dm,cs.ds,cs.et,cs.fl,cs.gl,cs.gr,cs.gt,cs.hc,cs.ir,cs.it,cs.lg,cs.lo,cs.ma,cs.mm,cs.ms,cs.na,cs.ne,cs.ni,cs.oh,cs.os,cs.pf,cs.pl,cs.ro,cs.sc,cs.sd,cs.se,cs.si,cs.sy,econ.em,econ.gn,econ.th,eess.as,eess.iv,eess.sp,eess.sy,math.ac,math.ag,math.ap,math.at,math.ca,math.co,math.ct,math.cv,math.dg,math.ds,math.fa,math.gm,math.gn,math.gr,math.gt,math.ho,math.it,math.kt,math.lo,math.mg,math.mp,math.na,math.nt,math.oa,math.oc,math.pr,math.qa,math.ra,math.rt,math.sg,math.sp,math.st,nlin.ao,nlin.cd,nlin.cg,nlin.ps,nlin.si,physics.acc-ph,physics.ao-ph,physics.app-ph,physics.atm-clus,physics.atom-ph,physics.bio-ph,physics.chem-ph,physics.class-ph,physics.comp-ph,physics.data-an,physics.ed-ph,physics.flu-dyn,physics.gen-ph,physics.geo-ph,physics.hist-ph,physics.ins-det,physics.med-ph,physics.optics,physics.plasm-ph,physics.pop-ph,physics.soc-ph,physics.space-ph,q-bio.bm,q-bio.cb,q-bio.gn,q-bio.mn,q-bio.nc,q-bio.ot,q-bio.pe,q-bio.qm,q-bio.sc,q-bio.to,q-fin.cp,q-fin.ec,q-fin.gn,q-fin.mf,q-fin.pm,q-fin.pr,q-fin.rm,q-fin.st,q-fin.tr,stat.ap,stat.co,stat.me,stat.ml,stat.ot,stat.th,submitter_label,doi_id_label,author_first_label,pub_publisher_label,license_label,category_main_label,category_main_detail_label,category_name_parent_label,category_name_parent_main_label,category_name_label,doi_cites_mean_author_first_label,doi_cites_count_author_first_label,doi_cites_sum_author_first_label,doi_cites_min_author_first_label,doi_cites_max_author_first_label,doi_cites_median_author_first_label,doi_cites_std_author_first_label,doi_cites_q10_author_first_label,doi_cites_q25_author_first_label,doi_cites_q75_author_first_label,doi_cites_mean_doi_id_label,doi_cites_count_doi_id_label,doi_cites_sum_doi_id_label,doi_cites_min_doi_id_label,doi_cites_max_doi_id_label,doi_cites_median_doi_id_label,doi_cites_std_doi_id_label,doi_cit

In [4]:
dorop_cols = []
for col in df_train.columns:
    if '_x' in col:
        dorop_cols.append(col)
len(dorop_cols), dorop_cols
df_train = df_train.drop(dorop_cols, axis=1)


In [5]:
NFOLDS = 10
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=777)

df_train['fold_no'] = [0 for x in range(len(df_train))]
for fold, (train_idx, val_idx) in enumerate(kf.split(df_train)):
    df_train.loc[val_idx, 'fold_no'] = fold
df_train.head()


0.090434         0.001255        -0.047360        -0.103646   

   roberta_vec_587  roberta_vec_588  roberta_vec_589  roberta_vec_590  \
0         0.041590        12.282816         0.079741        -0.060993   
1         0.033566        12.149046         0.052788        -0.019711   
2        -0.036725        11.943683         0.009125        -0.021048   
3         0.084762        11.718389        -0.038374        -0.011906   
4         0.108661        11.651844        -0.063221        -0.013029   

   roberta_vec_591  roberta_vec_592  roberta_vec_593  roberta_vec_594  \
0         0.063440        -0.268049        -0.094870         0.024665   
1         0.077738        -0.249763        -0.025224         0.030602   
2         0.045698        -0.227533        -0.049069        -0.027898   
3        -0.001632        -0.224642        -0.079878        -0.099328   
4         0.032770        -0.256653         0.004424        -0.158540   

   roberta_vec_595  roberta_vec_596  roberta_vec_597  roberta_vec_598  \
0         0.001880         0.187180         0.040667         0.006068   
1        -0.058953         0.241092         0.107833        -0.050415   
2         0.021803         0.141909         0.143262         0.028343   
3         0.124719         0.179040         0.142606        -0.036361   
4         0.057062         0.279952         0.185350        -0.026612   

   roberta_vec_599  roberta_vec_600  roberta_vec_601  roberta_vec_602  \
0         0.023697         0.187934        -0.221596         0.068942   
1        -0.096370         0.116405        -0.205613         0.103472   
2         0.013603         0.211610        -0.150498        -0.003177   
3        -0.080203         0.120659        -0.049413         0.000616   
4        -0.073181         0.144206        -0.092292         0.073268   

   roberta_vec_603  roberta_vec_604  roberta_vec_605  roberta_vec_606  \
0         0.005736         0.098046         0.084538         0.091436   
1         0.020359         0.087780         0.165174         0.151956   
2         0.041957        -0.032630         0.211481         0.146104   
3         0.114762         0.136759         0.128589         0.083282   
4         0.087679         0.119163         0.097052         0.128711   

   roberta_vec_607  roberta_vec_608  roberta_vec_609  roberta_vec_610  \
0        -0.074916         0.082979         0.174427        -0.010615   
1        -0.155135         0.048719         0.136277         0.014289   
2        -0.185986         0.156548         0.081175         0.050995   
3        -0.119280         0.175465         0.175211         0.079239   
4        -0.163158         0.196666         0.096923         0.021693   

   roberta_vec_611  roberta_vec_612  roberta_vec_613  roberta_vec_614  \
0         0.956542        -0.157765         0.202592        -0.065107   
1         0.941260        -0.143702         0.072925        -0.135912   
2         0.899738        -0.161586         0.089926        -0.130796   
3         0.902776        -0.046504         0.255126        -0.002165   
4         0.768567        -0.057968         0.151456        -0.043386   

   roberta_vec_615  roberta_vec_616  roberta_vec_617  roberta_vec_618  \
0         0.086034         0.082509         0.116933        -0.130661   
1         0.126020         0.117271         0.201530        -0.107007   
2        -0.123190         0.086665         0.235653        -0.042723   
3        -0.024777         0.089408         0.198304         0.016981   
4        -0.004558         0.144182         0.187076         0.072016   

   roberta_vec_619  roberta_vec_620  roberta_vec_621  roberta_vec_622  \
0         0.056648        -0.001675        -0.010225         0.085153   
1         0.218389         0.065998        -0.083668         0.053277   
2         0.308070         0.091798        -0.021499        -0.014432   
3         0.230631         0.072909        -0.130362        -0.013177   
4         0.245870         0.112004        -0.171178       

In [6]:
result_y_test = []
result_y_valid = []
lgb_result_proba_test = []
lgb_result_proba_valid = []


lgb_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.1,
    'feature_pre_filter': False,
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'min_child_samples': 20,
    'num_iterations': 2000,
    'early_stopping_round': 50,
}

for fold_no in range(NFOLDS):
    test_fold_no = fold_no
    valid_fold_no = fold_no + 1
    if valid_fold_no == NFOLDS:
        valid_fold_no = 0

    # train
    train = df_train.copy()
    y_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]['cites'].values
    y_valid = train[train['fold_no'] == valid_fold_no]['cites'].values
    y_test = train[train['fold_no'] == test_fold_no]['cites'].values

    train = train.drop(
        ['id', 'authors', 'title', 'comments',
        'journal-ref', 'doi', 'report-no', 'categories', 'license',
        'abstract', 'versions', 'authors_parsed', 'pub_publisher',
        'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique',
        'submitter_label','doi_id_label','author_first_label','pub_publisher_label',
        'license_label','category_main_label','category_name_parent_label','category_name_parent_main_label', 'category_name_label'
        ]
        , axis=1
    )
    x_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]
    x_valid = train[train['fold_no'] == valid_fold_no]
    x_test = train[train['fold_no'] == test_fold_no]

    '''
    # target encoding
    target = 'doi_cites'
    key = 'pred_doi_cites'
    x_train, x_valid, x_test = make_statics_table(target, key, 'doi_cites', x_train, x_valid, x_test)

    keys = ['update_ym']
    target = 'cites'
    for key in keys:
        x_train, x_valid, x_test = make_statics_table(target, key, 'doi_cites', x_train, x_valid, x_test)
    '''

    # drop
    x_train = x_train.drop(['cites', 'fold_no'], axis=1)
    x_valid = x_valid.drop(['cites', 'fold_no'], axis=1)
    x_test = x_test.drop(['cites', 'fold_no'], axis=1)

    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(x_train, y_train,
                eval_set=(x_valid, y_valid),
                eval_metric='rmse',
                verbose=100,
                early_stopping_rounds=20,
    )

    feature = x_train.copy()
    if fold_no == 0:
        df_feature = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), index=feature.columns, columns=['importance_0']).reset_index().rename(columns={'index':'name'}).reset_index()
    else:
        df_temp = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), index=feature.columns, columns=[f'importance_{fold_no}']).reset_index().rename(columns={'index':'name'}).reset_index().drop('index', axis=1)
        df_feature = pd.merge(df_feature, df_temp, on='name', how='left')

    fold_result_test = model.predict(x_test)
    fold_result_valid = model.predict(x_valid)
    result_y_valid.extend(y_valid)
    result_y_test.extend(y_test)
    lgb_result_proba_test.extend(fold_result_test)
    lgb_result_proba_valid.extend(fold_result_valid)
    rmsle = mean_squared_error(y_test, fold_result_test, squared=False)
    print(f"fold {fold_no} lgb score: {rmsle}")


print(mean_squared_error(result_y_valid, lgb_result_proba_valid, squared=False))
print(mean_squared_error(result_y_test, lgb_result_proba_test, squared=False))


/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 0.48836
Early stopping, best iteration is:
[65]	valid_0's rmse: 0.487616
fold 0 lgb score: 0.4872573431155

In [ ]:
score: 0.5020611604729619

In [7]:
df_feature['sum'] = (df_feature['importance_0'] + df_feature['importance_1'] + df_feature['importance_2'] + df_feature['importance_3'] + df_feature['importance_4'] + df_feature['importance_5'] + df_feature['importance_6'] + df_feature['importance_7'] + df_feature['importance_8'] + df_feature['importance_9'])

df_feature['sum'] = df_feature['sum'] / df_feature['sum'].sum()
df_feature.sort_values('sum',ascending=False).head(50)

,index,name,importance_0,importance_1,importance_2,importance_3,importance_4,importance_5,importance_6,importance_7,importance_8,importance_9,sum
836,836,diff_doi_cites_mean_category_main_label_pred_d...,12816.429386,4263.872318,3394.700451,12438.496852,12011.525717,8178.835996,15306.647201,11578.764069,3324.666420,13576.846773,0.144089
498,498,diff_doi_cites_doi_cites_mean_category_main_label,15581.979277,1412.579711,2865.635499,2141.886474,9348.296500,2049.931542,2117.450247,5523.559901,18344.022143,5771.014480,0.096896
524,524,diff_doi_cites_pred_doi_cites_mean_category_ma...,6510.448655,10348.366129,7381.594965,3221.942901,7955.561375,2914.178700,15212.983203,3727.892071,3664.663053,3376.872515,0.095644
500,500,diff_doi_cites_doi_cites_mean_category_main_de...,4785.385463,370.323503,7425.468496,8434.294341,1513.924914,10337.781034,1856.756042,8699.261459,11762.900350,6153.066110,0.091219
868,868,diff_doi_cites_mean_category_main_detail_label...,235.124001,14298.645408,14498.570855,5232.727819,5247.652431,3446.085498,6206.007819,1943.871261,39.419701,170.974254,0.076318
526,526,diff_doi_cites_pred_doi_cites_mean_category_ma...,1260.446991,8926.252621,7075.560615,7155.721725,2068.899902,11399.809989,671.621002,8071.978316,222.334000,154.186701,0.069905
499,499,diff_rate_doi_cites_doi_cites_mean_category_ma...,1302.739941,1709.466340,1349.180233,4519.162291,4427.906778,2150.879885,2742.915271,3966.053113,4597.921536,1894.456309,0.042622
496,496,diff_doi_cites_doi_cites_mean_license_label,1417.751553,1135.486362,1190.360491,1860.611632,1123.977873,3463.631494,2569.727172,2527.034163,1676.250688,2503.947787,0.028953
992,992,diff_pred_doi_cites_pred_doi_cites_mean_catego...,766.071199,1600.971317,0.000000,1861.229061,1967.561996,562.531877,617.078091,221.082993,747.686199,8256.377958,0.024687
0,0,doi_cites,903.117815,684.576007,1443.307099,1196.654010,845.241165,999.309256,1147.516272,2192.245653,2087.289368,3240.542746,0.021920


In [9]:
# bert vec一応残す
del_importance0_col = df_feature[df_feature['sum'] == 0]['name'].values
del_importance0_col

array(['econ_y', 'eess_y', 'nlin_y', 'physics_y', 'acc-phys_y',
       'adap-org_y', 'alg-geom_y', 'ao-sci_y', 'astro-ph_y', 'atom-ph_y',
       'bayes-an_y', 'chao-dyn_y', 'chem-ph_y', 'cmp-lg_y', 'comp-gas_y',
       'dg-ga_y', 'funct-an_y', 'gr-qc_y', 'math-ph_y', 'mtrl-th_y',
       'nucl-ex_y', 'patt-sol_y', 'plasm-ph_y', 'q-alg_y', 'q-fin_y',
       'solv-int_y', 'supr-con_y', 'acc_y', 'adap_y', 'alg_y', 'ao_y',
       'astro_y', 'atom_y', 'bayes_y', 'chao_y', 'chem_y', 'cmp_y',
       'comp_y', 'cond_y', 'dg_y', 'econ', 'eess', 'funct_y', 'gr_y',
       'math', 'mtrl_y', 'nlin', 'patt_y', 'physics', 'plasm_y',
       'quant_y', 'solv_y', 'stat', 'supr_y', 'astro-ph.ga',
       'astro-ph.he', 'astro-ph.sr', 'cond-mat.dis-nn',
       'cond-mat.mes-hall', 'cond-mat.other', 'cond-mat.soft',
       'cond-mat.stat-mech', 'cs.ai', 'cs.ar', 'cs.cc', 'cs.ce', 'cs.cg',
       'cs.cl', 'cs.cr', 'cs.cv', 'cs.cy', 'cs.db', 'cs.dc', 'cs.dl',
       'cs.dm', 'cs.et', 'cs.fl', 'cs.gl', 'cs.gr',

In [10]:
result_y = []
lgb_result_proba_test = []
lgb_result_proba_valid = []

lgb_params = {
    'objective': 'regression',
    'metric': 'rmse',
}

for fold_no in range(NFOLDS):
    test_fold_no = fold_no
    valid_fold_no = fold_no + 1
    if valid_fold_no == NFOLDS:
        valid_fold_no = 0

    # train
    train = df_train.copy()
    y_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]['cites'].values
    y_valid = train[train['fold_no'] == valid_fold_no]['cites'].values
    y_test = train[train['fold_no'] == test_fold_no]['cites'].values

    train = train.drop(
        ['id', 'authors', 'title', 'comments',
        'journal-ref', 'doi', 'report-no', 'categories', 'license',
        'abstract', 'versions', 'authors_parsed', 'pub_publisher',
        'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique']
        , axis=1
    )
    train = train.drop(del_importance0_col, axis=1)
    
    x_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]
    x_valid = train[train['fold_no'] == valid_fold_no]
    x_test = train[train['fold_no'] == test_fold_no]

    '''
    # target encoding
    target = 'doi_cites'
    key = 'pred_doi_cites'
    x_train, x_valid, x_test = make_statics_table(target, key, 'doi_cites', x_train, x_valid, x_test)

    keys = ['update_ym']
    target = 'cites'
    for key in keys:
        x_train, x_valid, x_test = make_statics_table(target, key, 'doi_cites', x_train, x_valid, x_test)
    '''

    # drop
    x_train = x_train.drop(['cites', 'fold_no'], axis=1)
    x_valid = x_valid.drop(['cites', 'fold_no'], axis=1)
    x_test = x_test.drop(['cites', 'fold_no'], axis=1)

    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(x_train, y_train,
                eval_set=(x_valid, y_valid),
                eval_metric='rmse',
                verbose=100,
                early_stopping_rounds=20,
    )

    feature = x_train.copy()
    if fold_no == 0:
        df_feature = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), index=feature.columns, columns=['importance_0']).reset_index().rename(columns={'index':'name'}).reset_index()
    else:
        df_temp = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), index=feature.columns, columns=[f'importance_{fold_no}']).reset_index().rename(columns={'index':'name'}).reset_index().drop('index', axis=1)
        df_feature = pd.merge(df_feature, df_temp, on='name', how='left')

    fold_result_test = model.predict(x_test)
    fold_result_valid = model.predict(x_valid)
    result_y.extend(y_test)
    lgb_result_proba_test.extend(fold_result_test)
    lgb_result_proba_valid.extend(fold_result_valid)
    rmsle = mean_squared_error(y_test, fold_result_test, squared=False)
    print(f"fold {fold_no} lgb score: {rmsle}")


rmsle = mean_squared_error(result_y, lgb_result_proba_test, squared=False)
print("+-" * 40)
print(f"score: {rmsle}")


/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[60]	valid_0's rmse: 0.489523
fold 0 lgb score: 0.4924097564526638
/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))
Training until validation scores don't improve for 20 rounds
Earl

In [11]:
df_feature['sum'] = (df_feature['importance_0'] + df_feature['importance_1'] + df_feature['importance_2'] + df_feature['importance_3'] + df_feature['importance_4'] + df_feature['importance_5'] + df_feature['importance_6'] + df_feature['importance_7'] + df_feature['importance_8'] + df_feature['importance_9'])

df_feature['sum'] = df_feature['sum'] / df_feature['sum'].sum()
df_feature.sort_values('sum',ascending=False).head(50)

,index,name,importance_0,importance_1,importance_2,importance_3,importance_4,importance_5,importance_6,importance_7,importance_8,importance_9,sum
580,580,diff_doi_cites_mean_category_main_label_pred_d...,31870.114937,24582.073644,32542.454114,23034.522594,15204.024879,19946.422881,23655.736117,11515.207665,16673.263716,31193.856256,0.283473
265,265,diff_doi_cites_doi_cites_mean_category_main_label,12918.882702,24973.310529,7186.648948,11840.174718,15342.412922,12732.207327,9312.111661,27714.145795,28073.172960,10823.677622,0.198141
290,290,diff_doi_cites_pred_doi_cites_mean_category_ma...,5838.796108,1790.435557,12291.003147,15035.926880,17067.570096,14576.259766,18972.683554,9912.648495,1176.043226,6007.446951,0.126419
266,266,diff_rate_doi_cites_doi_cites_mean_category_ma...,6683.923796,6676.251112,7361.090855,7821.532072,8250.093879,5209.459751,6127.151576,8235.577102,8414.882003,7483.062016,0.088979
263,263,diff_doi_cites_doi_cites_mean_license_label,1752.451809,1903.803786,1708.470609,4450.769109,4721.344572,3592.506208,2671.487234,2617.570805,2558.422016,1341.726163,0.033638
0,0,doi_cites,1824.477476,1636.997835,2383.664973,2928.773728,1982.227281,1580.990147,1716.599886,3550.634203,4958.362691,3765.712647,0.032419
68,68,category_name_label,1067.850302,1034.480223,1029.882769,1060.390369,1008.378698,1083.305606,1060.888564,1220.116659,1118.402090,1118.048052,0.013300
628,628,diff_doi_cites_mean_category_name_parent_label...,1791.762342,2624.233259,467.330925,287.932295,48.188499,1139.670570,153.926502,73.592060,3746.977422,117.322155,0.012869
269,269,diff_doi_cites_doi_cites_mean_category_name_pa...,1253.882289,1659.210713,2489.868732,116.570379,952.871458,97.967901,451.343080,471.291403,250.872962,2470.754642,0.012578
553,553,diff_doi_cites_mean_license_label_pred_doi_cites,1362.746657,842.337090,1169.822388,804.685602,652.204796,894.311499,519.320795,770.507811,1148.251427,1350.067050,0.011715


In [42]:
df_feature.sort_values('sum',ascending=False).tail(50)

,index,name,importance_0,importance_1,importance_2,importance_3,importance_4,importance_5,importance_6,importance_7,importance_8,importance_9,sum
217,217,diff_doi_cites_mean_pub_publisher_label_doi_ci...,2.57621,0.00000,7.09750,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000012
27,27,math_y,9.47646,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000012
223,223,diff_doi_cites_mean_update_ym_doi_cites_mean_c...,0.00000,3.26757,0.00000,2.205240,0.00000,3.98224,0.00000,0.00000,0.00000,0.00000,0.000012
211,211,diff_rate_doi_cites_mean_pub_publisher_label_d...,0.00000,0.00000,0.00000,2.491520,4.13107,0.00000,2.81075,0.00000,0.00000,0.00000,0.000012
732,732,roberta_vec_473,0.00000,0.00000,0.00000,0.000000,2.36340,0.00000,0.00000,0.00000,3.63331,3.33214,0.000011
70,70,doi_cites_count_pub_publisher_label,5.03146,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,4.27307,0.00000,0.00000,0.000011
236,236,diff_rate_doi_cites_mean_first_created_ym_doi_...,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,5.07053,0.00000,4.19324,0.00000,0.000011
244,244,diff_doi_cites_mean_license_label_doi_cites_me...,6.88974,0.00000,0.00000,2.201330,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000011
179,179,diff_rate_doi_cites_mean_doi_id_label_doi_cite...,0.00000,0.00000,2.65028,0.000000,4.12360,0.00000,0.00000,2.29863,0.00000,0.00000,0.000011
247,247,diff_doi_cites_mean_category_main_label_doi_ci...,0.00000,0.00000,0.00000,2.970610,0.00000,2.57544,0.00000,3.32829,0.00000,0.00000,0.000011


In [27]:
SEED = 777
result_y_test = []
result_y_valid = []
cat_result_proba_test = []
cat_result_proba_valid = []

for fold_no in range(NFOLDS):
    test_fold_no = fold_no
    valid_fold_no = fold_no + 1
    if valid_fold_no == NFOLDS:
        valid_fold_no = 0

    # train
    train = df_train.copy()
    y_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]['cites'].values
    y_valid = train[train['fold_no'] == valid_fold_no]['cites'].values
    y_test = train[train['fold_no'] == test_fold_no]['cites'].values

    train = train.drop(
        ['id', 'authors', 'title', 'comments',
        'journal-ref', 'doi', 'report-no', 'categories', 'license',
        'abstract', 'versions', 'authors_parsed', 'pub_publisher',
        'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique']
        , axis=1
    )

    x_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]
    x_valid = train[train['fold_no'] == valid_fold_no]
    x_test = train[train['fold_no'] == test_fold_no]

    # target encoding
    #target = 'doi_cites'
    #key = 'pred_doi_cites'
    #x_train, x_valid, x_test = make_statics_table(target, key, x_train, x_valid, x_test)

    #keys = ['update_ym', 'license_label', 'pub_publisher_label', 'category_main_label', 'category_name_parent_label', 'category_name_parent_main_label', 'category_name_label', 'pred_doi_cites']
    '''
    keys = ['update_ym', 'license_label', 'pub_publisher_label', 'category_main_label', 'category_name_parent_label', 'category_name_parent_main_label']
    target = 'cites'
    for key in keys:
        x_train, x_valid, x_test = make_statics_table(target, key, x_train, x_valid, x_test)
    '''
    
    # drop
    x_train = x_train.drop(['cites', 'fold_no'], axis=1)
    x_valid = x_valid.drop(['cites', 'fold_no'], axis=1)
    x_test = x_test.drop(['cites', 'fold_no'], axis=1)


    cat_features = np.where(x_train.dtypes == 'category')[0]
    train_pool = Pool(x_train, y_train, cat_features=cat_features)
    validate_pool = Pool(x_valid, y_valid, cat_features=cat_features)
    test_pool = Pool(x_test, y_test, cat_features=cat_features)
    
    model = CatBoostRegressor(loss_function='RMSE')
    #model = lgb.LGBMRegressor(**lgb_params)
    model.fit(train_pool,
                eval_set=validate_pool,
                verbose=100,
                early_stopping_rounds=20,
    )

    pickle.dump(model, open(f'../models/cat_{fold_no}.pickle', 'wb'))

    fold_result_test = model.predict(x_test)
    fold_result_valid = model.predict(x_valid)
    result_y_test.extend(y_test)
    result_y_valid.extend(y_valid)
    cat_result_proba_test.extend(fold_result_test)
    cat_result_proba_valid.extend(fold_result_valid)

    rmsle = mean_squared_error(y_test, fold_result_test, squared=False)
    print(f"fold {fold_no} lgb score: {rmsle}")

rmsle = mean_squared_error(result_y, cat_result_proba_test, squared=False)
print("+-" * 40)
print(f"score: {rmsle}")


Learning rate set to 0.073805
0:	learn: 1.1313618	test: 1.1293337	best: 1.1293337 (0)	total: 136ms	remaining: 2m 15s
100:	learn: 0.4873824	test: 0.4959874	best: 0.4959874 (100)	total: 5.85s	remaining: 52.1s
200:	learn: 0.4542212	test: 0.4903992	best: 0.4903689 (198)	total: 11.3s	remaining: 45.1s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.4884059369
bestIteration = 244

Shrink model to first 245 iterations.
fold 0 lgb score: 0.48712678916939334
Learning rate set to 0.073805
0:	learn: 1.1309215	test: 1.1066975	best: 1.1066975 (0)	total: 61.2ms	remaining: 1m 1s
100:	learn: 0.4848514	test: 0.5244766	best: 0.5244766 (100)	total: 6.04s	remaining: 53.8s
200:	learn: 0.4498289	test: 0.5194566	best: 0.5193126 (195)	total: 12s	remaining: 47.6s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.5176502565
bestIteration = 253

Shrink model to first 254 iterations.
fold 1 lgb score: 0.48503307629110604
Learning rate set to 0.073805
0:	learn: 1.1310489	test: 

In [53]:
df_result = pd.DataFrame([lgb_result_proba_test, cat_result_proba_test, result_y_test]).T.rename(columns={0:'lgb', 1:'cat', 2:'doi_cites', 3:'pred_doi_cites', 4:'y'})
df_result.head(50)

,lgb,cat,doi_cites
0,1.616322,1.668196,1.386294
1,1.150935,1.139835,3.663562
2,2.656537,2.606077,2.564949
3,2.070312,2.100779,1.791759
4,2.703382,2.741201,2.833213
5,2.786024,2.910353,2.995732
6,1.261919,1.377220,1.386294
7,1.120770,1.370008,1.609438
8,2.982970,2.812338,2.639057
9,1.282977,1.284003,1.386294


In [54]:
df_result.shape, len(result_y_test)

((15117, 3), 15117)

In [56]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model

X = df_result[['lgb', 'cat']] # , 'doi_cites', 'pred_doi_cites'
y = result_y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.25, random_state=0)
#model = linear_model.ElasticNet(alpha=0.7, l1_ratio=0.7, max_iter=5000)


In [57]:
model = linear_model.LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)

mean_squared_error(y_test, pred, squared=False)

0.49743021455230063

In [58]:
lgb_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.1,
    'feature_pre_filter': False,
    'num_leaves': 31,
    'feature_fraction': 0.8,
    'bagging_fraction': 1,
    'bagging_freq': 0,
    'min_child_samples': 20,
    'num_iterations': 2000,
    'early_stopping_round': 50,
}
model = lgb.LGBMRegressor(**lgb_params)
model.fit(X_train, y_train,
    eval_set=(X_test, y_test),
    eval_metric='rmse',
    verbose=100,
)

pred = model.predict(X_test)

mean_squared_error(y_test, pred, squared=False)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[34]	valid_0's rmse: 0.507305
/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


0.5073053637597862

In [28]:
x_train.columns[categorical_features_indices]

Index(['submitter_label', 'doi_id_label', 'author_first_label',
       'pub_publisher_label', 'license_label', 'category_main_label',
       'category_name_parent_label', 'category_name_parent_main_label',
       'category_name_label'],
      dtype='object')

In [ ]:
 # 0.5029752173028096 lgb normal

In [33]:
x_train.head()

,doi_cites,pub_journals,pub_dois,update_year,first_created_year,last_created_year,update_month,first_created_month,last_created_month,update_ym,first_created_ym,last_created_ym,update_day,first_created_day,last_created_day,update_date_unixtime,first_created_unixtime,last_created_unixtime,diff_update_date_unixtime,diff_created_unixtime,num_created,update_date_days,first_created_days,last_created_days,diff_created_days,rate_created_days,author_num,astro-ph.co,astro-ph.ep,astro-ph.ga,astro-ph.he,astro-ph.im,astro-ph.sr,cond-mat.dis-nn,cond-mat.mes-hall,cond-mat.mtrl-sci,cond-mat.other,cond-mat.quant-gas,cond-mat.soft,cond-mat.stat-mech,cond-mat.str-el,cond-mat.supr-con,cs.ai,cs.ar,cs.cc,cs.ce,cs.cg,cs.cl,cs.cr,cs.cv,cs.cy,cs.db,cs.dc,cs.dl,cs.dm,cs.ds,cs.et,cs.fl,cs.gl,cs.gr,cs.gt,cs.hc,cs.ir,cs.it,cs.lg,cs.lo,cs.ma,cs.mm,cs.ms,cs.na,cs.ne,cs.ni,cs.oh,cs.os,cs.pf,cs.pl,cs.ro,cs.sc,cs.sd,cs.se,cs.si,cs.sy,econ.em,econ.gn,econ.th,eess.as,eess.iv,eess.sp,eess.sy,math.ac,math.ag,math.ap,math.at,math.ca,math.co,math.ct,math.cv,math.dg,math.ds,math.fa,math.gm,math.gn,math.gr,math.gt,math.ho,math.it,math.kt,math.lo,math.mg,math.mp,math.na,math.nt,math.oa,math.oc,math.pr,math.qa,math.ra,math.rt,math.sg,math.sp,math.st,nlin.ao,nlin.cd,nlin.cg,nlin.ps,nlin.si,physics.acc-ph,physics.ao-ph,physics.app-ph,physics.atm-clus,physics.atom-ph,physics.bio-ph,physics.chem-ph,physics.class-ph,physics.comp-ph,physics.data-an,physics.ed-ph,physics.flu-dyn,physics.gen-ph,physics.geo-ph,physics.hist-ph,physics.ins-det,physics.med-ph,physics.optics,physics.plasm-ph,physics.pop-ph,physics.soc-ph,physics.space-ph,q-bio.bm,q-bio.cb,q-bio.gn,q-bio.mn,q-bio.nc,q-bio.ot,q-bio.pe,q-bio.qm,q-bio.sc,q-bio.to,q-fin.cp,q-fin.ec,q-fin.gn,q-fin.mf,q-fin.pm,q-fin.pr,q-fin.rm,q-fin.st,q-fin.tr,stat.ap,stat.co,stat.me,stat.ml,stat.ot,stat.th,acc-phys,adap-org,alg-geom,ao-sci,astro-ph,atom-ph,bayes-an,chao-dyn,chem-ph,cmp-lg,comp-gas,cond-mat,dg-ga,funct-an,gr-qc,hep-ex,hep-lat,hep-ph,hep-th,math-ph,mtrl-th,nucl-ex,nucl-th,patt-sol,plasm-ph,q-alg,q-bio,q-fin,quant-ph,solv-int,supr-con,acc,adap,alg,ao,astro,atom,bayes,chao,chem,cmp,comp,cond,cs,dg,econ,eess,funct,gr,hep,math,mtrl,nlin,nucl,patt,physics,plasm,q,quant,solv,stat,supr,submitter_label,doi_id_label,author_first_label,pub_publisher_label,license_label,category_main_label,doi_cites_mean_author_first_label,doi_cites_count_author_first_label,doi_cites_sum_author_first_label,doi_cites_min_author_first_label,doi_cites_max_author_first_label,doi_cites_median_author_first_label,doi_cites_std_author_first_label,doi_cites_q10_author_first_label,doi_cites_q25_author_first_label,doi_cites_q75_author_first_label,doi_cites_mean_doi_id_label,doi_cites_count_doi_id_label,doi_cites_sum_doi_id_label,doi_cites_min_doi_id_label,doi_cites_max_doi_id_label,doi_cites_median_doi_id_label,doi_cites_std_doi_id_label,doi_cites_q10_doi_id_label,doi_cites_q25_doi_id_label,doi_cites_q75_doi_id_label,doi_cites_mean_pub_publisher_label,doi_cites_count_pub_publisher_label,doi_cites_sum_pub_publisher_label,doi_cites_min_pub_publisher_label,doi_cites_max_pub_publisher_label,doi_cites_median_pub_publisher_label,doi_cites_std_pub_publisher_label,doi_cites_q10_pub_publisher_label,doi_cites_q25_pub_publisher_label,doi_cites_q75_pub_publisher_label,doi_cites_mean_submitter_label,doi_cites_count_submitter_label,doi_cites_sum_submitter_label,doi_cites_min_submitter_label,doi_cites_max_submitter_label,doi_cites_median_submitter_label,doi_cites_std_submitter_label,doi_cites_q10_submitter_label,doi_cites_q25_submitter_label,doi_cites_q75_submitter_label,doi_cites_mean_category_main_label,doi_cites_count_category_main_label,doi_cites_sum_category_main_label,doi_cites_min_category_main_label,doi_cites_max_category_main_label,doi_cites_median_category_main_label,doi_cites_std_category_main_label,doi_cites_q10_category_main_label,doi_cites_q25_category_main_label,doi_cites_q75_category_main_label,doi_cites_mean_update_ym,doi_cites_count_update_ym,doi_cites_sum

In [45]:
feature = x_train.copy()
df_feature = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), index=feature.columns, columns=['importance']).sort_values('importance', ascending=False)
df_feature.head(50)


,importance
diff_doi_cites_doi_cites_mean_category_main_label,45452.756427
te_diff_license_label_cites_mean,9845.287023
diff_rate_doi_cites_doi_cites_mean_category_main_label,9263.424620
te_rate_license_label_cites_mean,3971.043035
diff_doi_cites_doi_cites_mean_doi_id_label,1151.217911
doi_cites,696.934825
diff_doi_cites_doi_cites_mean_update_ym,596.695166
last_created_ym,393.102887
diff_rate_doi_cites_doi_cites_mean_pub_publisher_label,359.453805
category_main_label,283.692540


In [14]:
df_feature.tail(400)


,importance
rbert_vec_raw_768_665,0.0
rbert_vec_raw_768_697,0.0
cs.ds,0.0
rbert_vec_raw_768_695,0.0
rbert_vec_raw_768_693,0.0
...,...
rbert_vec_raw_768_341,0.0
bayes-an,0.0
atom-ph,0.0
rbert_vec_raw_768_344,0.0
